# Experiment Evaluation
Look at experiment results, compare them and find representative examples for the thesis motivation.

In [2]:
import os
from glob import glob
import pandas as pd

In [3]:
os.chdir("..")

In [12]:
os.getcwd()

'c:\\Users\\schla\\git\\enhancing-sgmcmc'

In [10]:
experiment_name = "unimodal_gaussian"

path = f"results/{experiment_name}/*/"
files = glob(path + "*.yaml")

df = pd.DataFrame()
for file in files:
    # flatten yaml and convert to dataframe (concat column names with _)
    df_temp = pd.read_yaml(file, flatten=True)
    df_temp = pd.json_normalize(df_temp, sep="_")
    # add file name as column
    df_temp["file"] = file.split("/")[-1]
    # add to dataframe
    df = pd.concat([df, df_temp], ignore_index=True)

df

""


In [11]:
files

[]